In [2]:
import numpy as np
import pandas as pd


In [12]:
df_all = pd.read_csv('bank/bank-full.csv', sep=';')
columns = [
    '年齡', '職業', '婚姻', '學歷', '違約', '平均餘額', '房屋貸款', '個人信貸', '聯絡方式', '最近一次通話日期', '最近一次通話月份', '最近一次通話秒數', 
    '通話次數_促銷期間', '上次促銷後_經過天數', '通話次數_促銷之前', '上次促銷結果', '本次促銷結果'
]
df_all.columns = columns
print(df_all.shape)
print(df_all['本次促銷結果'].value_counts())
rate = df_all['本次促銷結果'].value_counts()['yes']/len(df_all)
print(f'銷售成功率: {rate:.4f}')


(45211, 17)
本次促銷結果
no     39922
yes     5289
Name: count, dtype: int64
銷售成功率: 0.1170


In [13]:
print(df_all.isnull().sum())

年齡            0
職業            0
婚姻            0
學歷            0
違約            0
平均餘額          0
房屋貸款          0
個人信貸          0
聯絡方式          0
最近一次通話日期      0
最近一次通話月份      0
最近一次通話秒數      0
通話次數_促銷期間     0
上次促銷後_經過天數    0
通話次數_促銷之前     0
上次促銷結果        0
本次促銷結果        0
dtype: int64
